In [1]:
import pandas as pd
import positions as pt
import vasicek as vc
import valuation as vn

In [2]:
%run vasicek.ipynb
#%run data_usd_monthly.ipynb
#%run monthly_data.ipynb

2011-09-30T00:00:00.000000000
2011-10-31T00:00:00.000000000
2011-11-30T00:00:00.000000000
2011-12-30T00:00:00.000000000
2012-01-31T00:00:00.000000000
2012-02-29T00:00:00.000000000
2012-03-30T00:00:00.000000000
2012-04-30T00:00:00.000000000
2012-05-31T00:00:00.000000000
2012-06-29T00:00:00.000000000
2012-07-31T00:00:00.000000000
2012-08-31T00:00:00.000000000
2012-09-28T00:00:00.000000000
2012-10-31T00:00:00.000000000
2012-11-30T00:00:00.000000000
2012-12-31T00:00:00.000000000
2013-01-31T00:00:00.000000000
2013-02-28T00:00:00.000000000
2013-03-29T00:00:00.000000000
2013-04-30T00:00:00.000000000
2013-05-28T00:00:00.000000000
2013-06-28T00:00:00.000000000
2013-07-31T00:00:00.000000000
2013-08-29T00:00:00.000000000
2013-09-30T00:00:00.000000000
2013-10-31T00:00:00.000000000
2013-11-28T00:00:00.000000000
2013-12-31T00:00:00.000000000
2014-01-30T00:00:00.000000000
2014-02-28T00:00:00.000000000
2014-03-31T00:00:00.000000000
2014-04-30T00:00:00.000000000
2014-05-29T00:00:00.000000000
2014-06-30

In [ ]:
'''
model_swap_rates = pd.ExcelFile('fullperiod_modelrates.xlsx')
parameter_data = pd.ExcelFile('fullperiod_parameters.xlsx')
weight_data = pd.ExcelFile('fullperiod_weights.xlsx')

model_swap_rates = model_swap_rates.parse('Sheet1')
parameter_data = parameter_data.parse('Sheet1')
weight_data = weight_data.parse('Sheet1')

model_swap_rates = model_swap_rates.drop(['Unnamed: 0'], axis=1)
parameter_data = parameter_data.drop(['Unnamed: 0'], axis=1)
weight_data = weight_data.drop(['Unnamed: 0'], axis=1)
'''

In [10]:
# Cubic data for dirty price valuation
boot_ois = vn.bootstrap_ois(data1_ois, "Act/360")
cub_ois = vn.cubic_ois(boot_ois)
cub_swap = vn.cubic_swap(data1_swap_trade)
cub_libor = vn.cubic_libor(data1_libor)

In [11]:
# Adding extra YYYY/MM date column to the dataframes
data1_swap_trade['date2'] = data1_swap_trade['date'].dt.to_period('M')
data1_libor_3M_original['date2'] =  data1_libor_3M_original['date'].dt.to_period('M')
model_swap_rates['date2'] = model_swap_rates['date'].dt.to_period('M')
weight_data['date2'] = weight_data['date'].dt.to_period('M')

C:\Users\Miro_\AppData\Local\Temp/ipykernel_4392/1498460402.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1_libor_3M_original['date2'] =  data1_libor_3M_original['date'].dt.to_period('M')


In [1]:
# "Infrastructure" for creating the dictionary of books
dates = pd.date_range('2015-01', '2022-01', freq='M').to_period('M')
temp_book = pt.create_book()
book_dict = pt.create_dict_of_books()

NameError: name 'pd' is not defined

In [1]:
lev_multiplier = 16.327853889313875

In [2]:
closed_trades = []
for date in dates:
    model_size = model_swap_rates[model_swap_rates['date2'] == date].size
    swap_size = data1_swap_trade[data1_swap_trade['date2'] == date].size
    data1_libor_3M_original_size = data1_libor_3M_original[data1_libor_3M_original['date2'] == date].size
    weight_size = weight_data[weight_data['date2'] == date].size
    if model_size != 0 and swap_size != 0 and data1_libor_3M_original_size != 0 and weight_size != 0:
        long_date = data1_swap_trade[data1_swap_trade['date2']==date].values[0][0]
        temp_book = pt.update_book(temp_book, data1_libor_3M_original, data1_swap_trade, date)
        temp_book = pt.add_to_book(date, temp_book, model_swap_rates, data1_swap_trade, data1_libor_3M_original, weight_data, 0.0005, 100*lev_multiplier, 3, 6)
        temp_book = pt.update_current_dirty_prices(long_date, temp_book, cub_libor, cub_swap, cub_ois, "Act/360", "30/360", 3, 6)
        book_dict[date] = temp_book
        temp_book, closes_convergence = pt.clean_book_convergence(temp_book)
        temp_book, closes_maturity = pt.clean_book_maturity(temp_book)
        closed_trades.append([date, closes_convergence + closes_maturity])
        temp_book = pt.update_latest_dirty_prices(long_date, temp_book, cub_libor, cub_swap, cub_ois, "Act/360", "30/360", 3, 6)
    else:
        #long_date = data1_libor_3M_original[data1_libor_3M_original['date2']==date].values[0][0] # poista
        temp_book = pt.update_book2(temp_book, data1_libor_3M_original, date)
        #temp_book = pt.update_current_dirty_prices(long_date, temp_book, cub_libor, cub_swap, cub_ois, "Act/360", "30/360", 3, 6) # poista
        book_dict[date] = temp_book
        temp_book, closes_convergence = pt.clean_book_convergence(temp_book)
        temp_book, closes_maturity = pt.clean_book_maturity(temp_book)
        closed_trades.append([date, closes_convergence*3 + closes_maturity*2])
        #temp_book = pt.update_latest_dirty_prices(long_date, temp_book, cub_libor, cub_swap, cub_ois, "Act/360", "30/360", 3, 6) # poista
closed_trades_df = pd.DataFrame(closed_trades, columns=['date','closed count'])

NameError: name 'dates' is not defined